## Global Settings and parameters

In [1]:
import os
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator


In [2]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = 25
data_path = "../../Data/Amazon_Movie"

## Data Processing

### Process Dataset

In [3]:
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = 'reviews_Movies_and_TV_5.json'
meta_name = 'meta_Movies_and_TV.json'
reviews_file = os.path.join(data_path, reviews_name)
meta_file = os.path.join(data_path, meta_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
sample_rate = 0.01 # sample a small item set for training and testing here for fast example

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)

### Create data loader

In [4]:
input_creator = SequentialIterator

## Sli-Rec Model

Z. Yu, J. Lian, A. Mahmoody, G. Liu and X. Xie, "Adaptive User Modeling with Long and Short-Term Preferences for Personailzed Recommendation", in Proceedings of the 28th International Joint Conferences on Artificial Intelligence, IJCAI’19, Pages 4213-4219, AAAI Press, 2019.

### Prepare hyper-parameters

In [5]:
yaml_file = './sli_rec.yaml'
hparams = prepare_hparams(
    yaml_file,
    embed_l2=0.,
    layer_l2=0., 
    learning_rate=0.001,  # set to 0.01 if batch normalization is disable
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    show_step=20,
    user_vocab=user_vocab,
    item_vocab=item_vocab,
    cate_vocab=cate_vocab,
    need_sample=True,
    train_num_ngs=train_num_ngs # provides the number of negative instances for each positive instance for loss computation.
)

### Create Model

In [6]:
%%capture
from recommenders.models.deeprec.models.sequential.sli_rec import SLI_RECModel as SeqModel
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

### Initial Performance

In [7]:
sli_rec_test_result_before_training = model.run_eval(test_file, num_ngs=test_num_ngs)
sli_rec_test_result_before_training

{'auc': 0.5011,
 'logloss': 0.6931,
 'mean_mrr': 0.2782,
 'ndcg@2': 0.1463,
 'ndcg@6': 0.3114,
 'ndcg@10': 0.4426,
 'group_auc': 0.5015}

### Model Training

In [8]:
with Timer() as train_time:
    if os.path.exists("./model/slirec/"):
        model_path = os.path.join("./model/slirec/", "best_model")
        model.load_model(model_path)
    else:
        model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs, eval_metric=)

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

step 20 , total_loss: 1.6096, data_loss: 1.6096
step 40 , total_loss: 1.6047, data_loss: 1.6047
eval valid at epoch 1: auc:0.5135,logloss:0.6931,mean_mrr:0.4685,ndcg@2:0.3427,ndcg@6:0.5988,ndcg@10:0.5988,group_auc:0.5169
step 20 , total_loss: 1.5626, data_loss: 1.5626
step 40 , total_loss: 1.4966, data_loss: 1.4966
eval valid at epoch 2: auc:0.5995,logloss:0.6919,mean_mrr:0.5109,ndcg@2:0.4063,ndcg@6:0.6315,ndcg@10:0.6315,group_auc:0.5717
step 20 , total_loss: 1.4076, data_loss: 1.4076
step 40 , total_loss: 1.3575, data_loss: 1.3575
eval valid at epoch 3: auc:0.6735,logloss:0.7333,mean_mrr:0.5918,ndcg@2:0.5194,ndcg@6:0.6934,ndcg@10:0.6934,group_auc:0.6705
step 20 , total_loss: 1.3021, data_loss: 1.3021
step 40 , total_loss: 1.2554, data_loss: 1.2554
eval valid at epoch 4: auc:0.7189,logloss:0.6249,mean_mrr:0.6354,ndcg@2:0.5745,ndcg@6:0.7263,ndcg@10:0.7263,group_auc:0.7089
step 20 , total_loss: 1.2412, data_loss: 1.2412
step 40 , total_loss: 1.2948, data_loss: 1.2948
eval valid at epoch 

In [9]:
# %%capture

# sli_rec_training_result = []
# for i in range(EPOCHS):
#     epoch_model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
#     model_path = os.path.join("./model/slirec/", "epoch_" + str(i + 1))
#     epoch_model.load_model(model_path)
#     print('loading saved model in {0}'.format(model_path))
#     sli_rec_training_result.append((i + 1, epoch_model.run_eval(valid_file, num_ngs=valid_num_ngs)))

### Performance After Training

In [10]:
sli_rec_test_result_after_training = model.run_eval(test_file, num_ngs=test_num_ngs)
sli_rec_test_result_after_training

{'auc': 0.7177,
 'logloss': 0.5588,
 'mean_mrr': 0.4862,
 'ndcg@2': 0.3986,
 'ndcg@6': 0.5526,
 'ndcg@10': 0.6077,
 'group_auc': 0.7066}

## A2SVD Model

Z. Yu, J. Lian, A. Mahmoody, G. Liu and X. Xie, "Adaptive User Modeling with Long and Short-Term Preferences for Personailzed Recommendation", in Proceedings of the 28th International Joint Conferences on Artificial Intelligence, IJCAI’19, Pages 4213-4219, AAAI Press, 2019.

### Prepare Hyper-parameters

In [11]:
yaml_file = './asvd.yaml'
hparams = prepare_hparams(
    yaml_file,
    embed_l2=0.,
    layer_l2=0., 
    learning_rate=0.001,  # set to 0.01 if batch normalization is disable
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    show_step=20,
    user_vocab=user_vocab,
    item_vocab=item_vocab,
    cate_vocab=cate_vocab,
    need_sample=True,
    train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
)

### Create Model

In [12]:
%%capture

from recommenders.models.deeprec.models.sequential.asvd import A2SVDModel as SeqModel
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

### Initial Performance

In [13]:
a2svd_test_result_before_training = model.run_eval(test_file, num_ngs=test_num_ngs)
a2svd_test_result_before_training

{'auc': 0.5027,
 'logloss': 0.6931,
 'mean_mrr': 0.2783,
 'ndcg@2': 0.1467,
 'ndcg@6': 0.3107,
 'ndcg@10': 0.4426,
 'group_auc': 0.5018}

### Model Training

In [14]:
with Timer() as train_time:
    if os.path.exists("./model/asvd/"):
        model_path = os.path.join("./model/asvd/", "best_model")
        model.load_model(model_path)
    else:
        model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

step 20 , total_loss: 1.6081, data_loss: 1.6081
step 40 , total_loss: 1.6038, data_loss: 1.6038
eval valid at epoch 1: auc:0.4953,logloss:0.6931,mean_mrr:0.4511,ndcg@2:0.3177,ndcg@6:0.5855,ndcg@10:0.5855,group_auc:0.4945
step 20 , total_loss: 1.5984, data_loss: 1.5984
step 40 , total_loss: 1.5614, data_loss: 1.5614
eval valid at epoch 2: auc:0.5057,logloss:0.6889,mean_mrr:0.4558,ndcg@2:0.3259,ndcg@6:0.5889,ndcg@10:0.5889,group_auc:0.4959
step 20 , total_loss: 1.5203, data_loss: 1.5203
step 40 , total_loss: 1.4824, data_loss: 1.4824
eval valid at epoch 3: auc:0.6579,logloss:0.6493,mean_mrr:0.57,ndcg@2:0.4862,ndcg@6:0.6768,ndcg@10:0.6768,group_auc:0.6438
step 20 , total_loss: 1.3980, data_loss: 1.3980
step 40 , total_loss: 1.3700, data_loss: 1.3700
eval valid at epoch 4: auc:0.6847,logloss:0.5602,mean_mrr:0.603,ndcg@2:0.5344,ndcg@6:0.702,ndcg@10:0.702,group_auc:0.6812
step 20 , total_loss: 1.3265, data_loss: 1.3265
step 40 , total_loss: 1.3484, data_loss: 1.3484
eval valid at epoch 5: au

In [15]:
# %%capture

# a2svd_training_result = []
# for i in range(EPOCHS):
#     epoch_model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
#     model_path = os.path.join("./model/asvd", "epoch_" + str(i + 1))
#     epoch_model.load_model(model_path)
#     print('loading saved model in {0}'.format(model_path))
#     a2svd_training_result.append((i + 1, epoch_model.run_eval(valid_file, num_ngs=valid_num_ngs)))

### Performance After Training

In [16]:
a2svd_test_result_after_training = model.run_eval(test_file, num_ngs=test_num_ngs)
a2svd_test_result_after_training

{'auc': 0.6884,
 'logloss': 0.5128,
 'mean_mrr': 0.4601,
 'ndcg@2': 0.3656,
 'ndcg@6': 0.5227,
 'ndcg@10': 0.587,
 'group_auc': 0.68}

## Caser Model

J. Tang and K. Wang, "Personalized top-n sequential recommendation via convolutional sequence embedding", in Proceedings of the Eleventh ACM International Conference on Web Search and Data Mining, ACM, 2018.

### Prepare Hyper-parameters

In [17]:
yaml_file = './caser.yaml'
hparams = prepare_hparams(
    yaml_file,
    embed_l2=0.,
    layer_l2=0., 
    learning_rate=0.001,  # set to 0.01 if batch normalization is disable
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    show_step=20,
    user_vocab=user_vocab,
    item_vocab=item_vocab,
    cate_vocab=cate_vocab,
    need_sample=True,
    train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
)

### Create Model

In [18]:
%%capture

from recommenders.models.deeprec.models.sequential.caser import CaserModel as SeqModel
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

### Initial Performance

In [19]:
caser_test_result_before_training = model.run_eval(test_file, num_ngs=test_num_ngs)
caser_test_result_before_training

{'auc': 0.4918,
 'logloss': 0.6931,
 'mean_mrr': 0.2718,
 'ndcg@2': 0.141,
 'ndcg@6': 0.3033,
 'ndcg@10': 0.4374,
 'group_auc': 0.4911}

### Model Training

In [20]:
with Timer() as train_time:
    if os.path.exists("./model/caser/"):
        model_path = os.path.join("./model/caser/", "best_model")
        model.load_model(model_path)
    else:
        model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs)

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

step 20 , total_loss: 1.6100, data_loss: 1.6100
eval valid at epoch 1: auc:0.4957,logloss:0.6929,mean_mrr:0.4503,ndcg@2:0.3156,ndcg@6:0.5849,ndcg@10:0.5849,group_auc:0.4959
step 20 , total_loss: 1.5630, data_loss: 1.5630
eval valid at epoch 2: auc:0.5943,logloss:0.692,mean_mrr:0.5262,ndcg@2:0.4194,ndcg@6:0.643,ndcg@10:0.643,group_auc:0.5853
step 20 , total_loss: 1.4401, data_loss: 1.4401
eval valid at epoch 3: auc:0.6365,logloss:0.6949,mean_mrr:0.5655,ndcg@2:0.4807,ndcg@6:0.6732,ndcg@10:0.6732,group_auc:0.6354
step 20 , total_loss: 1.3128, data_loss: 1.3128
eval valid at epoch 4: auc:0.6321,logloss:0.7347,mean_mrr:0.5664,ndcg@2:0.4765,ndcg@6:0.6737,ndcg@10:0.6737,group_auc:0.6333
step 20 , total_loss: 1.2964, data_loss: 1.2964
eval valid at epoch 5: auc:0.6661,logloss:0.7785,mean_mrr:0.5975,ndcg@2:0.526,ndcg@6:0.6975,ndcg@10:0.6975,group_auc:0.6685
step 20 , total_loss: 1.1570, data_loss: 1.1570
eval valid at epoch 6: auc:0.6821,logloss:0.7907,mean_mrr:0.6071,ndcg@2:0.529,ndcg@6:0.7046

In [21]:
# %%capture

# caser_training_result = []
# for i in range(EPOCHS):
#     epoch_model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
#     model_path = os.path.join("./model/caser/", "epoch_" + str(i + 1))
#     epoch_model.load_model(model_path)
#     print('loading saved model in {0}'.format(model_path))
#     sli_rec_training_result.append((i + 1, epoch_model.run_eval(valid_file, num_ngs=valid_num_ngs)))

### Performance After Training

In [22]:
caser_test_result_after_training = model.run_eval(test_file, num_ngs=test_num_ngs)
caser_test_result_after_training

{'auc': 0.6962,
 'logloss': 0.6784,
 'mean_mrr': 0.4646,
 'ndcg@2': 0.3702,
 'ndcg@6': 0.5276,
 'ndcg@10': 0.5906,
 'group_auc': 0.686}